In [55]:
"""
functions to compute the coordinates of a canoe given its parameters and visualize a canoe
"""
import numpy as np
import plotly.graph_objects as go
from ipywidgets import *

def get_coordinates(w, h, l, c, a, b, canoe_type):
    """
    get the 3D coordinates of the canoe
    input: 4 floating point numbers for width, height, length, and corner radius of the canoe
    input: 2 list of floating point numbers for eta, and theta
    input: 1 integer for canoe type
    output: list of x-coordinates of the canoe, named X
    output: list of y-coordinates of the canoe, named Y
    output: list of z-coordinates of the canoe, named Z
    """
    X=x_coordinates(l, a)
    Y=y_coordinates(w, c, a, b, canoe_type)
    Z=z_coordinates(h, c, a, b, canoe_type)
    return X, Y, Z

def x_coordinates(l, a):
    """
    input: length of canoe, and eta
    output: list of x-coordinates of the canoe
    """
    return l*np.cos(a)

def y_coordinates(w, c, a, b, canoe_type):
    """
    input: width of canoe, corner radius of canoe, eta, theta, and canoe type
    output: list of y-coordinates of the canoe
    """
    if canoe_type==3:
        return w*np.sin(a)*np.abs(np.sin(b))**(c/(w*np.sin(a)))*np.sign(np.sin(b))
    else:
        return w*np.sin(a)*np.sin(b)

def z_coordinates(h, c, a, b, canoe_type):
    """
    input: height of canoe, corner radius of canoe, eta, theta, and canoe type
    output: list of z-coordinates of the canoe
    """
    if canoe_type==1:
        return h*np.cos(b)
    if canoe_type==2:
        return h*np.abs(np.sin(a))*np.cos(b)
    else:
        return h*np.abs(np.sin(a))*np.abs(np.cos(b))**(c/(h*np.abs(np.sin(a))))*np.sign(np.cos(b))
    
def get_eta_theta(canoe_type):
    """
    input: canoe type
    output: list of angles in radians called eta and theta
    """
    if canoe_type==3:
        a, b=np.meshgrid(np.linspace(0.01, np.pi-0.01, 50), np.linspace(np.pi/2, (3/2)*np.pi, 50))
    else:
        a, b=np.meshgrid(np.linspace(0.0, 2*np.pi, 50), np.linspace(np.pi/2, (3/2)*np.pi, 50))
    return a, b

def plot_canoe(width, height, length, corner_radius, canoe_type):
    """
    make a 3D figure of a canoe. 
    input: 1 integer for canoe type
    input: 4 lists of floating point numbers showing ranges of width, 
        ranges of height, ranges of length, and ranges of corner radius for the canoe
    output: 3D figure of the canoe with 4 sliders for width, height, length, and corner radius
    """

    eta, theta=get_eta_theta(canoe_type)
    X, Y, Z=get_coordinates(width[0], height[0], length[0], corner_radius[0], eta, theta, canoe_type)
    fig=go.Figure()
    fig.add_trace(go.Surface(x=X, y=Y, z=Z, visible=False, showscale=False))
    """
    make the title, define range of x,y,z axis, define name of x,y,z axis,
    bound y-axis by x-axis, and make sure that figure of canoe always show up in the center
    """
    max_range=max(width[1], height[1], length[1])
    min_range=(-max_range)
    fig.layout=go.Layout(title='Type '+str(canoe_type)+' canoe visualization',
        scene=dict(
        xaxis=dict(range=[-length[1], length[1]]), xaxis_title='Length(m)',
        yaxis=dict(range=[-width[1], width[1]]), yaxis_title='Width(m)',
        zaxis=dict(range=[-height[1], 1]), zaxis_title='Height(m)'))
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.data[0].visible=True
    """
    make a slider for width, height, length, and corner radius. Each slider slides by 0.1. 
    calculate coordinates for new figure as slider slides, and show that figure
    """
    @interact(width=(width[0], width[1], 0.1), height=(height[0], height[1], 0.1), 
              length=(length[0], length[1], 0.1), corner_radius=(corner_radius[0], corner_radius[1], 0.1))
    def update(width=width[1], height=height[1], length=length[1], corner_radius=corner_radius[1]):
        with fig.batch_update():
            X, Y, Z=get_coordinates(width, height, length, corner_radius, eta, theta, canoe_type)
            fig.data[0].x=X
            fig.data[0].y=Y
            fig.data[0].z=Z
        return fig

## Define canoe type, width range, height range, length range, and corner radius range.

#### Canoe Type
There are 3 canoe types. 

Canoe type 1 approximates the cross-section of the canoe in width direction and height direction as a half-ellipse, and approximates the cross-section of the top part of the canoe in width direction and length direction as an ellipse.

Canoe type 2 makes same approximation as canoe type 1 for the cross-section of the canoe in width direction and height direction. This canoe type approximates the cross-section of the canoe in width direction and length direction as a half-ellipse.

Canoe type 3 approximate the longitudinal cross-section of the canoe as a half rectircle (rectangle with circular corners) with a given corner radius for width direction and length direction. This canoe type approximates the cross-section of the canoe in width direction and length direction as a half-ellipse. 

#### Height
Top part of the canoe shows up at 0m point. Function makes the canoe with given height input.
#### Width
Canoe is centered at 0m point. Given an input of Xm in width, the canoe has width of Xm in positive and negative directions.
#### Length
Canoe is centered at 0m point. Given an input of Xm in length, the canoe has width of Xm in positive and negative directions.
#### Corner Radius
This number is only used for canoe type 3. 

In [59]:
import panel as pn
pn.extension()
slider=[0,0,0,0,0];
slider[0]=pn.widgets.IntSlider(name='Type', start=1, end=3)
slider[1]=pn.widgets.RangeSlider(name='Width Range', start=1, end=10, step=0.1)
slider[2]=pn.widgets.RangeSlider(name='Height Range', start=1, end=10, step=0.1)
slider[3]=pn.widgets.RangeSlider(name='Length Range', start=1, end=10, step=0.1)
slider[4]=pn.widgets.RangeSlider(name='Corner Radius Range', start=1, end=10, step=0.1)
display(slider[0], slider[1], slider[2], slider[3], slider[4])

IntSlider(end=3, name='type', start=1, value=1)

RangeSlider(end=10, name='Width Range', start=1, value=(1, 10), value_end=10, value_start=1)

RangeSlider(end=10, name='Height Range', start=1, value=(1, 10), value_end=10, value_start=1)

RangeSlider(end=10, name='Length Range', start=1, value=(1, 10), value_end=10, value_start=1)

RangeSlider(end=10, name='Corner Radius Range', start=1, value=(1, 10), value_end=10, value_start=1)

In [64]:
canoe_type=slider[0].value
width=[slider[1].value[0], slider[1].value[1]]
height=[slider[2].value[0], slider[2].value[1]]
length=[slider[3].value[0], slider[3].value[1]]
corner_radius=[slider[4].value[0], slider[4].value[1]]
plot_canoe(width, height, length, corner_radius, canoe_type)

interactive(children=(FloatSlider(value=10.0, description='width', max=10.0, min=1.0), FloatSlider(value=10.0,…